## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page <font color='red'> For Seller to update:[Title_of_your_product](Provide link to your marketplace listing of your product).</font>
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

# Background

- **Model**: `ner_vop_pipeline`

- **Model Description**: This pipeline includes the full taxonomy Named-Entity Recognition model to extract information from health-related text in colloquial language. This pipeline extracts diagnoses, treatments, tests, anatomical references and demographic entities.




In [1]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [2]:
import base64
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
import boto3
from IPython.display import Image, display
from PIL import Image as ImageEdit
import numpy as np

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [4]:
model_name = "jsl-ner-vop-pipeline"

content_type = "application/json"

real_time_inference_instance_type = "ml.m4.xlarge"
batch_transform_inference_instance_type = "ml.m4.xlarge"


### A. Create an endpoint

In [5]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

---------------!

Once endpoint has been created, you would be able to perform real-time inference.

In [6]:
import json
import pandas as pd
import os
import boto3


# Set display options
pd.set_option('display.max_rows', None)  
pd.set_option('display.max_columns', None)  
pd.set_option('display.max_colwidth', None) 


def process_data_and_invoke_realtime_endpoint(data_dicts):
    for data_dict in data_dicts:
        json_input_data = json.dumps(data_dict)    
        i = 1
        input_file_name = f'inputs/real-time/input{i}.json'
        output_file_name = f'outputs/real-time/out{i}.out'
        
        while os.path.exists(input_file_name) or os.path.exists(output_file_name):
            i += 1
            input_file_name = f'inputs/real-time/input{i}.json'
            output_file_name = f'outputs/real-time/out{i}.out'

        os.makedirs(os.path.dirname(input_file_name), exist_ok=True)
        os.makedirs(os.path.dirname(output_file_name), exist_ok=True)
        
        with open(input_file_name, 'w') as f:
            f.write(json_input_data)
        
        s3_client.put_object(Bucket=s3_bucket, Key=f"validation-input-json/{os.path.basename(input_file_name)}", Body=bytes(json_input_data.encode('UTF-8')))
        
        response = sm_runtime.invoke_endpoint(
            EndpointName=model_name,
            ContentType=content_type,
            Accept="application/json",
            Body=json_input_data,
        )

        # Process response
        response_data = json.loads(response["Body"].read().decode("utf-8"))

        df = pd.DataFrame(response_data['predictions'], columns=['document', 'ner_chunk', 'begin', 'end', 'ner_label', 'confidence'])
        display(df)

        # Save response data to file
        with open(output_file_name, 'w') as f_out:
            json.dump(response_data, f_out, indent=4)


  **Input format**:
  
  
  {"**text**": "Refers to the complete text being analyzed by the pipeline, encompassing all content subjected to named entity recognition."}



In [7]:
data_dicts = [ { "text" :["Maria is a 20-year-old girl who was diagnosed with hyperthyroidism 1 month ago.",
    "She is admitted to The John Hopkins Hospital 2 days ago with a history of gestational diabetes mellitus diagnosed. She denied pain and any headache."]}
             ]
process_data_and_invoke_realtime_endpoint(data_dicts)


,document,ner_chunk,begin,end,ner_label,confidence
0,Maria is a 20-year-old girl who was diagnosed with hyperthyroidism 1 month ago.,20-year-old,11,21,Age,0.9618
1,Maria is a 20-year-old girl who was diagnosed with hyperthyroidism 1 month ago.,girl,23,26,Gender,0.9991
2,Maria is a 20-year-old girl who was diagnosed with hyperthyroidism 1 month ago.,hyperthyroidism,51,65,Disease,0.9743
3,Maria is a 20-year-old girl who was diagnosed with hyperthyroidism 1 month ago.,1 month ago,67,77,DateTime,0.664
4,She is admitted to The John Hopkins Hospital 2 days ago with a history of gestational diabetes mellitus diagnosed. She denied pain and any headache.,She,0,2,Gender,1.0
5,She is admitted to The John Hopkins Hospital 2 days ago with a history of gestational diabetes mellitus diagnosed. She denied pain and any headache.,admitted,7,14,AdmissionDischarge,0.9999
6,She is admitted to The John Hopkins Hospital 2 days ago with a history of gestational diabetes mellitus diagnosed. She denied pain and any headache.,John Hopkins Hospital,23,43,ClinicalDept,0.6984666
7,She is admitted to The John Hopkins Hospital 2 days ago with a history of gestational diabetes mellitus diagnosed. She denied pain and any headache.,2 days ago,45,54,DateTime,0.8039
8,She is admitted to The John Hopkins Hospital 2 days ago with a history of gestational diabetes mellitus diagnosed. She denied pain and any headache.,gestational diabetes mellitus,74,102,Disease,0.6344333
9,She is admitted to The John Hopkins Hospital 2 days ago with a history of gestational diabetes mellitus diagnosed. She denied pain and any headache.,She,115,117,Gender,1.0


In [8]:
data_dicts = [
    {"text" : """Mike, who is suffering from a rare form of cancer known as glioblastoma, recently underwent genomic sequencing that revealed a mutation in the TP53 gene, a key regulator in the cell cycle and apoptosis, often associated with tumor development."""},
]

process_data_and_invoke_realtime_endpoint(data_dicts)

,document,ner_chunk,begin,end,ner_label,confidence
0,"Mike, who is suffering from a rare form of cancer known as glioblastoma, recently underwent genomic sequencing that revealed a mutation in the TP53 gene, a key regulator in the cell cycle and apoptosis, often associated with tumor development.",cancer,43,48,Disease,0.9695
1,"Mike, who is suffering from a rare form of cancer known as glioblastoma, recently underwent genomic sequencing that revealed a mutation in the TP53 gene, a key regulator in the cell cycle and apoptosis, often associated with tumor development.",glioblastoma,59,70,Disease,0.9747
2,"Mike, who is suffering from a rare form of cancer known as glioblastoma, recently underwent genomic sequencing that revealed a mutation in the TP53 gene, a key regulator in the cell cycle and apoptosis, often associated with tumor development.",recently,73,80,DateTime,0.9734
3,"Mike, who is suffering from a rare form of cancer known as glioblastoma, recently underwent genomic sequencing that revealed a mutation in the TP53 gene, a key regulator in the cell cycle and apoptosis, often associated with tumor development.",genomic sequencing,92,109,Test,0.51305
4,"Mike, who is suffering from a rare form of cancer known as glioblastoma, recently underwent genomic sequencing that revealed a mutation in the TP53 gene, a key regulator in the cell cycle and apoptosis, often associated with tumor development.",TP53 gene,143,151,Test,0.5085
5,"Mike, who is suffering from a rare form of cancer known as glioblastoma, recently underwent genomic sequencing that revealed a mutation in the TP53 gene, a key regulator in the cell cycle and apoptosis, often associated with tumor development.",apoptosis,192,200,Disease,0.7065
6,"Mike, who is suffering from a rare form of cancer known as glioblastoma, recently underwent genomic sequencing that revealed a mutation in the TP53 gene, a key regulator in the cell cycle and apoptosis, often associated with tumor development.",tumor,225,229,Disease,0.9577


### C. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [9]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 3. Batch inference

In [10]:
validation_input_path = f"s3://{s3_bucket}/validation-input-json/"
validation_output_path = f"s3://{s3_bucket}/validation-output-json/"

In [11]:
input_file_name = 'inputs/batch/input.json'
output_file_name = 'outputs/batch/out.out'

os.makedirs(os.path.dirname(input_file_name), exist_ok=True)
os.makedirs(os.path.dirname(output_file_name), exist_ok=True)

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type="ml.m4.xlarge",
    accept="application/json",
)
transformer.transform(validation_input_path, content_type=content_type)
transformer.wait()

In [13]:
validation_file_name = "input.json"
input_json_data = {"text" :["Maria is a 20-year-old girl who was diagnosed with hyperthyroidism 1 month ago.",
    "She is admitted to The John Hopkins Hospital 2 days ago with a history of gestational diabetes mellitus diagnosed. She denied pain and any headache."]}
json_input_data = json.dumps(input_json_data)
with open(input_file_name, 'w') as f:
    f.write(json_input_data)

In [14]:
from urllib.parse import urlparse

parsed_url = urlparse(transformer.output_path)
file_key = f"{parsed_url.path[1:]}/{validation_file_name}.out"
response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

data = json.loads(response["Body"].read().decode("utf-8"))
df = pd.DataFrame(data['predictions'], columns=['document', 'ner_chunk', 'begin', 'end', 'ner_label', 'confidence'])
display(df)

with open(output_file_name, 'w') as f_out:
    json.dump(data, f_out, indent=4)

,document,ner_chunk,begin,end,ner_label,confidence
0,Maria is a 20-year-old girl who was diagnosed with hyperthyroidism 1 month ago.,20-year-old,11,21,Age,0.9618
1,Maria is a 20-year-old girl who was diagnosed with hyperthyroidism 1 month ago.,girl,23,26,Gender,0.9991
2,Maria is a 20-year-old girl who was diagnosed with hyperthyroidism 1 month ago.,hyperthyroidism,51,65,Disease,0.9743
3,Maria is a 20-year-old girl who was diagnosed with hyperthyroidism 1 month ago.,1 month ago,67,77,DateTime,0.664
4,She is admitted to The John Hopkins Hospital 2 days ago with a history of gestational diabetes mellitus diagnosed. She denied pain and any headache.,She,0,2,Gender,1.0
5,She is admitted to The John Hopkins Hospital 2 days ago with a history of gestational diabetes mellitus diagnosed. She denied pain and any headache.,admitted,7,14,AdmissionDischarge,0.9999
6,She is admitted to The John Hopkins Hospital 2 days ago with a history of gestational diabetes mellitus diagnosed. She denied pain and any headache.,John Hopkins Hospital,23,43,ClinicalDept,0.6984666
7,She is admitted to The John Hopkins Hospital 2 days ago with a history of gestational diabetes mellitus diagnosed. She denied pain and any headache.,2 days ago,45,54,DateTime,0.8039
8,She is admitted to The John Hopkins Hospital 2 days ago with a history of gestational diabetes mellitus diagnosed. She denied pain and any headache.,gestational diabetes mellitus,74,102,Disease,0.6344333
9,She is admitted to The John Hopkins Hospital 2 days ago with a history of gestational diabetes mellitus diagnosed. She denied pain and any headache.,She,115,117,Gender,1.0


In [15]:
model.delete_model()

INFO:sagemaker:Deleting model with name: jsl-ner-vop-pipeline-2024-02-27-19-08-21-757


### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

